# my voki tester notebook

In [2]:
import datetime
import numpy as np
import pandas as pd
import os
from pathlib import Path
import pickle

In [209]:
class voki_db:
    def __init__(self, df=None):
        pass


def card_id(card):
    return card['id'][0]


class vokibox():
    def __init__(self, n, max_cards=50):  # box_db,voki_db

        self.levels = {i: [] for i in range(n)}
        self.max_cards = max_cards
        self.n_levels=n

    def onboard_card(self, _id,level=0):
        if level >= self.n_levels:
            print(f'this vokibox has only {self.n_levels} levels, so you may specify only up to level index {self.n_levels-1}. you wanted to onboard a card to level index {level}, ')
            return
        
        if self.n_ids() >= self.max_cards:
            print(f'cannot onboard card {_id} as the maximum number of cards ({self.n_ids}) is already at the maximum allowed cards.')
            return
        
        if isinstance(_id, pd.DataFrame):
            _id = card_id(_id)
            
        if not _id in [c_id for lvl in self.levels.values() for c_id in lvl]:
            self.levels[level].append(_id)
        else:
            print(f'card id {_id} already in the box ')

    def attempt_card_in_level(self, lvl, df):
        # attempt the next card in vokibox level i
        if len(self.levels) <= lvl:
            print(f'You cannot attempt level {lvl} as there are only levels 0-{len(self.levels) - 1}')
            return False
        if len(self.levels[lvl]) == 0:
            print(f'vokibox lvl {lvl} is empty. you need to fill it first')
            return

        top_card_id = self.levels[lvl].pop(0)

        card, result = attempt(top_card_id, df)
        
        if result:
            self.advance(card, lvl)
        else:
            self.onboard_card(top_card_id)

    def advance(self, card, lvl):
        if lvl < len(self.levels) - 1:
            self.levels[lvl + 1].append(card['id'])
        else:
            print(f'congrats! card \"{card["french_phrase"]}\" made it through the vokibox!')

    def n_ids(self):
        return len([_id for level in self.levels.values() for _id in level])

    def __repr__(self):
        strlist = []
        for (i, lvl) in self.levels.items():
            strlist.append(f'level {i}')
            for id_ in lvl:
                strlist.append(f'    {id_}')

        return '\n'.join(strlist)


def box_db_load(box_db_file):
    with open(box_db_file, 'rb') as fh:
        return pickle.load(fh)


def box_db_save(box_db_file):
    with open(box_db_file, 'wb') as fh:
        pickle.dump(fh)


def card_from_id(ids, df):
    if not isinstance(ids, list):
        ids = [ids]
    return_cards = df[df['id'].isin(ids)]
    if len(return_cards)==0:
        print(f'ids {ids} not existing in df')
        
    return return_cards.iloc[0].to_dict()


def attempt(id_, df):
    card = card_from_id(id_, df)
    now = datetime.datetime.now().replace(microsecond=0)
    guess = input(f'{card["german_phrase"]}')
    result = check_attempt(card, guess)  # true/False

    card['attempt_history'] = card['attempt_history'].append(pd.DataFrame({'time': [now],
                                                                           'result': [result],
                                                                           'answer': [guess]}))
    if result:
        card['streak'] += 1
    else:
        card['streak'] = np.floor(card['streak'] / 2)
    return card, result


def check_attempt(card, guess):
    if guess == card['french_phrase']:
        print('correct!')
        return True
    text = input(f'type "yes" if you think they match:\n {guess} \n {card["french_phrase"]}\n')
    if text == 'yes':
        return True
    return False

In [211]:
# some testing
card_1 = pd.DataFrame({'french_phrase': ['partout'],
                            'french_sentences': [['Dans la rue du Sentier, la mode est partout.']],
                            'french_comments': [[]],
                            'german_phrase': ['überall'],
                            'german_sentences': [[]],
                            'german_comments': [[]],
                            'created': [datetime.datetime.now().replace(microsecond=0)],
                            'attempt_history': [pd.DataFrame({'time': pd.Series(dtype='datetime64[ns]'),
                                                             'result': pd.Series(dtype='bool'),
                                                             'answer': pd.Series(dtype='str')})],
                            'streak': [0],
                            'id': [hash('partout')]
                            })

card_2 = pd.DataFrame({'french_phrase': ['la rue'],
                            'french_sentences': [['Dans la rue du Sentier, la mode est partout.']],
                            'french_comments': [[]],
                            'german_phrase': ['Strasse'],
                            'german_sentences': [[]],
                            'german_comments': [[]],
                            'created': [datetime.datetime.now().replace(microsecond=0)],
                            'attempt_history': [pd.DataFrame({'time': pd.Series(dtype='datetime64[ns]'),
                                                             'result': pd.Series(dtype='bool'),
                                                             'answer': pd.Series(dtype='str')})],
                            'streak': [0],
                            'id': [hash('la rue')]
                            })
df = pd.concat([card_1,card_2])
myvokibox = vokibox(5)
myvokibox.onboard_card(card_1,level=4)
myvokibox.onboard_card(card_2)
myvokibox

level 0
    442165381189763710
level 1
level 2
level 3
level 4
    -4980411815809341467

In [213]:
myvokibox.attempt_card_in_level(4,df);

überall partout


correct!
congrats! card "partout" made it through the vokibox!


In [189]:
myvokibox.attempt_card_in_level(0,df)

überall partout


correct!


In [170]:
myvokibox

level 0
    442165381189763710
level 1
level 2
level 3
level 4
level 5
    -4980411815809341467

In [165]:
myvokibox.attempt_card_in_level(1,df)

vokibox lvl 1 is empty. you need to fill it first


In [166]:
myvokibox.attempt_card_in_level(2,df)

überall partout


correct!


In [167]:
myvokibox.attempt_card_in_level(3,df)

überall partout


correct!


In [169]:
myvokibox.attempt_card_in_level(4,df)

überall partout


correct!


In [171]:
myvokibox.attempt_card_in_level(5,df)

überall partout


correct!
congrats! card partout made it through thevokibox!


In [172]:
myvokibox

level 0
    442165381189763710
level 1
level 2
level 3
level 4
level 5

In [140]:
class card:
    # a class for vocabulary cards

    def __init__(self, props):
        # the basic card that has typical properties of a vocabulary card
        self.french_phrase = props['french']['phrase']
        self.french_sentences =      props['french']['sentences']
        self.french_comments = props['french']['comments']
        self.german_phrase = props['german']['phrase']
        self.german_sentences = props['german']['sentences']
        self.german_comments = props['german']['comments']

        self.created = datetime.datetime.now().replace(microsecond=0)
        self.attempt_history = pd.DataFrame({'time': pd.Series(dtype='datetime64[ns]'),
                                             'result': pd.Series(dtype='bool'),
                                             'answer': pd.Series(dtype='str')})
        self.id = hash(self.french_phrase)
        self.streak = 0

    def attempt(self, guess):
        now = datetime.datetime.now().replace(microsecond=0)
        result = self.check_attempt(guess) #true/False

        self.attempt_history = self.attempt_history.append(pd.DataFrame({'time': [now], 
                                                                        'result': [result], 
                                                                        'answer': [guess]}))
        if result:
            self.streak += 1
        else:
            self.streak = np.floor(self.streak/2)

    def check_attempt(self,answer):
        if answer == self.french_phrase:
            return True
        text = input(f'type "yes" if you think they match:\n {answer} \n {self.french_phrase}\n')
        if text == 'yes':
            return True
        return False
    
    def __repr__(self):
        
        if len(self.attempt_history)>0:
            last_attempt = ''
        str_ = f"""voki card 
        {self.id}
        french: {self.french_phrase}
        german: {self.german_phrase} 
        last_attempt: """
        return str_
    
    


    @staticmethod
    def get_card_from_db(db,id):
        pass
    

        
    



        

        
def main(dbfile):
    #1 laod database
    #2 run program
    #3 save databas
    pass



In [185]:
mycard = card_from_id(4980411815809341467,df)

IndexError: single positional indexer is out-of-bounds

In [177]:
mycard

{'french_phrase': 'la rue',
 'french_sentences': ['Dans la rue du Sentier, la mode est partout.'],
 'french_comments': [],
 'german_phrase': 'Strasse',
 'german_sentences': [],
 'german_comments': [],
 'created': Timestamp('2021-10-31 21:20:50'),
 'attempt_history': Empty DataFrame
 Columns: [time, result, answer]
 Index: [],
 'streak': 0,
 'id': 442165381189763710}

In [179]:
pd.DataFrame({key: [value] for key,value in mycard.items()})

,french_phrase,french_sentences,french_comments,german_phrase,german_sentences,german_comments,created,attempt_history,streak,id
0,la rue,"[Dans la rue du Sentier, la mode est partout.]",[],Strasse,[],[],2021-10-31 21:20:50,"Empty DataFrame Columns: [time, result, answer...",0,442165381189763710


In [170]:
root = Path.cwd()
data_dir = root.parent / 'data'
voki_db_file = data_dir / 'voki_db.pkl'
box_db_file = data_dir / 'box_db.pkl'

# main()

# df_voki = pd.read_pickle(voki_db_file)
# voki_box = box_db_load(box_db_file)

In [161]:
voki_box

level 0
    5116320665103601376
    463857425755945451
level 1
level 2
level 3
level 4
level 5

In [162]:
df_voki

,french_phrase,french_sentences,french_comments,german_phrase,german_sentences,created,attempt_history,id,streak
0,partout,"[Dans la rue du Sentier, la mode est partout.]",[],überall,[],2021-10-31 13:48:03,"Empty DataFrame Columns: [time, result, answer...",463857425755945451,0
0,la rue,[Julien habite dans la rue Daguerre.],[],Strasse,[],2021-10-31 13:48:04,"Empty DataFrame Columns: [time, result, answer...",5116320665103601376,0


In [115]:
my_vbox.onboard_card(voki_card1.id)
my_vbox

level 0
    5116320665103601376
    463857425755945451
level 1
level 2
level 3
level 4
level 5

In [121]:
box_db_load(box_db_file)

level 0
    5116320665103601376
    463857425755945451
level 1
level 2
level 3
level 4
level 5

In [3]:
card_1_dict = {'french_phrase': 'partout', 
                'french_sentences': ['Dans la rue du Sentier, la mode est partout.'],
                'french_comments':[],
                'german_phrase' :'überall',
                'german_sentences':[],
                'german_comments':[],
                'created': datetime.datetime.now().replace(microsecond=0),
                'attempt_history' : pd.DataFrame({'time': pd.Series(dtype='datetime64[ns]'),
                                             'result': pd.Series(dtype='bool'),
                                             'answer': pd.Series(dtype='str')})
                'streak': 0
                'id': hash('partout')
              }
                     
card_2_properties = {'french':{'phrase': 'la rue', 
                               'sentences': ['Julien habite dans la rue Daguerre.'],
                               'comments':[]},
                     'german':{'phrase':'Strasse',
                               'sentences':[],
                               'comments':[]},
                    }   

In [168]:
a=[[]]
a

[[]]

In [4]:
voki_card1=card(card_1_properties)
voki_card1

voki card 
        463857425755945451
        french: partout
        german: überall 
        last_attempt: 

In [5]:
voki_card2=card(card_2_properties)
voki_card2

voki card 
        5116320665103601376
        french: la rue
        german: Strasse 
        last_attempt: 

In [91]:
voki_card1.attempt('partout2'                  )


type "yes" if you think they match:
 partout2 
 partout no


In [92]:
voki_card1.attempt_history

,time,result,answer
0,2021-10-31 08:58:40,1.0,partout
0,2021-10-31 09:17:37,1.0,partout
0,2021-10-31 09:17:40,0.0,partout2


In [16]:
pd.DataFrame(voki_card1.to_df())

TypeError: to_df() got an unexpected keyword argument 'index'

In [19]:
df_list = [pd.DataFrame(voki_card1.to_df()),
           pd.DataFrame(voki_card2.to_df())]
df_voki = pd.concat(df_list)

In [32]:
df_voki.to_pickle(voki_db_file)

In [26]:
df_voki

,french_phrase,french_sentences,french_comments,german_phrase,german_sentences,created,attempt_history,id,streak
0,partout,"[Dans la rue du Sentier, la mode est partout.]",[],überall,[],2021-10-31 13:48:03,"Empty DataFrame Columns: [time, result, answer...",463857425755945451,0
0,la rue,[Julien habite dans la rue Daguerre.],[],Strasse,[],2021-10-31 13:48:04,"Empty DataFrame Columns: [time, result, answer...",5116320665103601376,0


In [35]:
a=pd.read_pickle(voki_db_file)
a

,french_phrase,french_sentences,french_comments,german_phrase,german_sentences,created,attempt_history,id,streak
0,partout,"[Dans la rue du Sentier, la mode est partout.]",[],überall,[],2021-10-31 13:48:03,"Empty DataFrame Columns: [time, result, answer...",463857425755945451,0
0,la rue,[Julien habite dans la rue Daguerre.],[],Strasse,[],2021-10-31 13:48:04,"Empty DataFrame Columns: [time, result, answer...",5116320665103601376,0
